In [1]:
from PIL import ImageDraw, Image, ImageFont

def code_to_img(text):
    img = Image.new('RGB', (400, 800))
    d = ImageDraw.Draw(img)
    # Specify the font size
    d.text((20, 20), text, fill=(255, 255, 255))
    return img

# code_to_img("Hello World")

In [2]:
from datasets import load_dataset
ds = load_dataset("codeparrot/github-code", streaming=True, split="train")
ds

/home/Ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


IterableDataset({
    features: ['code', 'repo_name', 'path', 'language', 'license', 'size'],
    n_shards: 1126
})

In [3]:
responses = []  # codes
images = []  # images of the codes
questions = []  # questions

count = 5000
for row in ds:
    if count % 100 == 0:
        print(count, end=" ")
    if count == 0:
        break
    code = row["code"][:1700]
    responses.append(code)
    images.append(code_to_img(code))
    questions.append("What's in this image?")
    count -= 1

len(responses), len(images), len(questions)

5000 4900 4800 4700 4600 4500 4400 4300 4200 4100 4000 3900 3800 3700 3600 3500 3400 3300 3200 3100 3000 2900 2800 2700 2600 2500 2400 2300 2200 2100 2000 1900 1800 1700 1600 1500 1400 1300 1200 1100 1000 900 800 700 600 500 400 300 200 100 0 

(5000, 5000, 5000)

In [4]:
# # Download image_url to PIL Image
# from PIL import Image
# import requests
# image_url = "https://i.ytimg.com/vi/71Z-Jpnm3D4/maxresdefault.jpg"

# def url_to_PIL(image_url):
#     img = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
#     return img

# print(url_to_PIL(image_url))

# # create dataset with custom columns
# from datasets import Dataset
# import numpy as np
# import pandas as pd

# questions = [
#     "What algorithm is portrayed in this image?",
#     "What is this image about?",
#     "What type of tree is in this image?",
#     "What algorithm is this diagram about?",
#     "What algorithm is this code using?"
# ]

# urls = [
#     "https://i.ytimg.com/vi/71Z-Jpnm3D4/maxresdefault.jpg",
#     "https://i.ytimg.com/vi/jsmMtJpPnhU/maxresdefault.jpg",
#     "https://web.cecs.pdx.edu/~sheard/course/Cs163/Graphics/FullBinary.jpg",
#     "https://i.ytimg.com/vi/XWo_Q0YiNY4/maxresdefault.jpg",
#     "https://www.andrew.cmu.edu/course/15-121/lectures/Sorting%20Algorithms/pix/bubbleSort.bmp"
# ]

# responses = [
#     "Dijkstra's Algorithm",
#     "Prim's Minimum Spanning Tree Algorithm",
#     "Binary Tree",
#     "Kosaraju's Algorithm",
#     "Sorting Algorithm",
# ]

# images = []
# for u in urls:
#     print(u)
#     images.append(url_to_PIL(u))

# len(questions), len(urls), len(images), len(responses)

In [5]:
from datasets import Dataset
data = {"question": questions, "image": images, "multiple_choice_answer": responses}
ds = Dataset.from_dict(data)
split_ds = ds.train_test_split(test_size=0.2)
train_ds = split_ds["train"]
print(train_ds[0])

{'question': "What's in this image?", 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=400x800 at 0x7FF15FD98C40>, 'multiple_choice_answer': "import { expect } from 'chai';\n\nimport { Curry } from './curry';\n\ndescribe('curry', () => {\n  it('should curry the method with default arity', () => {\n    class MyClass {\n      @Curry()\n      add(a: any, b?: any) {\n        return a + b;\n      }\n    }\n\n    const myClass = new MyClass();\n    const add5 = myClass.add(5);\n\n    expect(add5).to.be.an.instanceOf(Function);\n    expect(add5(10)).to.equal(15);\n  });\n\n  it('should curry the method with default arity (paramless)', () => {\n    class MyClass {\n      @Curry\n      add(a: any, b?: any) {\n        return a + b;\n      }\n    }\n\n    const myClass = new MyClass();\n    const add5 = myClass.add(5);\n\n    expect(add5).to.be.an.instanceOf(Function);\n    expect(add5(10)).to.equal(15);\n  });\n\n  it('should curry the method with fixed arity', () => {\n    class My

### Load Model

In [6]:
import os
from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration, BitsAndBytesConfig, TrainingArguments, Trainer
import torch
from peft import get_peft_model, LoraConfig
import dotenv
dotenv.load_dotenv()

# 2. Load Model
model_id = "google/paligemma-3b-pt-224"
processor = PaliGemmaProcessor.from_pretrained(model_id)
device = "cuda"

# Loading Quantised Model (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_type=torch.bfloat16
)
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj",
                    "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = PaliGemmaForConditionalGeneration.from_pretrained(model_id,
                                                          quantization_config=bnb_config,
                                                          device_map={"": 0})
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Unused kwargs: ['bnb_4bit_compute_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.18s/it]


trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


In [7]:
# 3. Fine-tuning the model
def collate_fn(examples):
    texts = ["answer " + example["question"] for example in examples]
    labels = [example['multiple_choice_answer'] for example in examples]
    images = [example["image"].convert("RGB") for example in examples]
    tokens = processor(text=texts, images=images, suffix=labels,
                       return_tensors="pt", padding="longest",
                       tokenize_newline_separately=False)
    tokens = tokens.to(torch.bfloat16).to(device)
    return tokens

args = TrainingArguments(
    num_train_epochs=2,
    remove_unused_columns=False,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=2,
    learning_rate=2e-5,
    weight_decay=1e-6,
    adam_beta2=0.999,
    logging_steps=100,
    optim="adamw_hf",
    save_strategy="steps",
    save_steps=1000,
    push_to_hub=True,
    save_total_limit=1,
    output_dir="paligemma_CS",
    bf16=True,
    dataloader_pin_memory=False
)

trainer = Trainer(
    model=model,
    train_dataset=train_ds,
    data_collator=collate_fn,
    args=args
)

trainer.train()

/home/Ubuntu/.local/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/Ubuntu/.local/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
100,1.592100
200,1.236600
300,1.177400
400,1.150000
500,1.157000


TrainOutput(global_step=500, training_loss=1.2626302490234376, metrics={'train_runtime': 2873.6138, 'train_samples_per_second': 2.784, 'train_steps_per_second': 0.174, 'total_flos': 8.456728320081274e+16, 'train_loss': 1.2626302490234376, 'epoch': 2.0})

In [8]:
# 4. Save the model in HuggingFace
trainer.push_to_hub('DJPAUL25/paligemma_CS')  # Ensure the pushed version is the final model

CommitInfo(commit_url='https://huggingface.co/DJPAUL25/paligemma_CS/commit/30aff425a4ddff40ec100aeace45374171408c76', commit_message='DJPAUL25/paligemma_CS', commit_description='', oid='30aff425a4ddff40ec100aeace45374171408c76', pr_url=None, pr_revision=None, pr_num=None)